In [1]:
%matplotlib inline

In [2]:
from Bio import SeqIO

# Notes and scattered thoughts

I wanted to get some alignments to play with so rather than using BLAST or associated types of programs, I went straight to the source: orthology databases. Most of the hard work has already been done for us in various databases especially with proteins as well-studed as Initiation and Elongation Factors.

**OMA Groups**

These alignments were downloaded from OMA, which provides high confidence ortholog groups. OMA groups are generally considered to be of the highest quality, but they are very conservative so the alignments that they provide are generally not huge/deep. But you can be confident that within the groups themselves the data is very reliable. I searched for genes/groups using uniprot identifiers for *E. coli* (IF2: `P0A705`; EF-Tu: `P0CE47`) and found some reasonably sized alignments:

1. https://omabrowser.org/oma/omagroup/593610/msa/

2. https://omabrowser.org/oma/omagroup/593581/msa/


Selecting HOGs rather than OMA groups will result in many more species but might be slightly messier:

1. https://omabrowser.org/oma/hog/HOG:0683840/Bacteria/msa/ 
    
    (note that the above alignment only stretches back to the common ancestor of Bacteria)


2. https://omabrowser.org/oma/hog/HOG:0711629/LUCA/msa/ 
    
    (whereas this alignment stretches back to LUCA)
    
An annoying aspect of all databases, of course, is that some species are dramatically over-represented. OMAGroups are not immune to this, and there are a lot of *E. coli* strains that are included in these alignments. Not a huge problem, but something to note. 

**eggNOG**

A slightly less accurate but far more expansive option would be to use eggnog (searching for the relevant uniprot identifiers):
1. http://eggnog5.embl.de/#/app/home 

    Large (>5000 sequences) alignments are found for each going back to the ancestral LUCA root. However, at first glance the genes are not full orthologs (i.e. both tufA and tufB are included in the alignment for E. coli, and are of course paralogs). Some work might need to be done to identify single copy orthologs per species. One nice thing, however, is that the `fasta` records use NCBI taxonomy identifiers making it pretty easy to parse these alignment files.
    
    Currently, these files reside in `../Data/AJH/` along with the OMAGroup data. But I am not using them...yet. 

# Identify the species that appear in both ortholog sets

In [3]:
###Read in the fasta files, note that OMA groups come PRE-aligned (and I'd like to do my own alignment)
if2_records = list(SeqIO.parse('../Data/AJH/Orthologs/OMAGroup_IF2.aln', 'fasta'))
eftu_records = list(SeqIO.parse('../Data/AJH/Orthologs/OMAGroup_EFTU.aln', 'fasta'))

In [4]:
if2_species = [record.id[:5] for record in if2_records] ###First 5 letters are the organism identifier
eftu_species = [record.id[:5] for record in eftu_records]
print('Species in IF2, EFTU, and the intersection:', len(if2_species),
                                                      len(eftu_species), 
                                                      len(set(if2_species).intersection(eftu_species)))
print()
valid_species = list(set(if2_species).intersection(eftu_species))
print('Number of species in both datasets (should match the above intersection):', len(valid_species))
print()
print('Found E. coli?:', 'ECOLI' in valid_species) ###Making sure our reference species is here

Species in IF2, EFTU, and the intersection: 1272 535 349

Number of species in both datasets (should match the above intersection): 349

Found E. coli?: True


# Clean the IF2 file to keep only the (newly defined) valid species

Also, I'm truncating the 5' end of the IF2 orthologs according to the sequence `EPRAPVVTIM` and removing all gap characters as I plan to align the sequences myself.

Note that we *could* stick with the original OMA alignments, which are probably fine? We'd just want to remove any columns that are all gaps now that we have truncated the overall number of species. And then doing a combined alignment with IF2 and EFTU would require seeding the alignments (more on that below).

In [5]:
###This just identifies where I want the new IF2 alignment to start from within the existing alignment
truncation_sequence = 'EPRAPVVTIM'

In [6]:
new_align_start = []
for record in if2_records:
    if record.id[:5] == 'ECOLI':
        print('E coli ID and description:', record.id, record.description)
        eco_seq = str(record.seq)
        ###This complicated little for loop tries to find the "truncation_sequence" while knowing
        ###that it might have gaps in it. So it temporarily removes gaps then checks if its the 
        ###truncation sequence that we define as the beginning of the domains we care abous
        for i in range(len(eco_seq)):
            if eco_seq[i] == '-':
                continue
            temp_seq = eco_seq[i:].replace('-', '')
            if temp_seq[:10] == truncation_sequence:
                new_align_start.append(i)
                
###Make sure that I found one (and only one) place where the truncation sequence was found
assert len(new_align_start) == 1
new_align_start = new_align_start[0]

###Now write the cleaned (and unaligned) files
with open('../Data/AJH/Orthologs/OMAGroup_IF2.clean.fa', 'w') as outfile:
    for record in if2_records:
        ###This simultaneousl truncates the original sequence, then removes the '-' characters
        ###because I ultimately want to align them myself
        clean_seq = str(record.seq)[new_align_start:].replace('-','') 
        
        ###Hopefully there are no super short sequences?
        if len(clean_seq) < 200:
            print('Really short sequence, this should not be happening...?')
            continue
            
        ###Write my new entries as a fasta format
        if record.id[:5] in valid_species:
            outfile.write('>{}\n'.format(record.id))
            outfile.write('{}\n'.format(clean_seq))

E coli ID and description: ECOLI02980 ECOLI02980


# And do the same for EFTU

In [7]:
###Same as above but I'm not truncating anything this time around and am keeping the entire EFTU seqs
with open('../Data/AJH/Orthologs/OMAGroup_EFTU.clean.fa', 'w') as outfile:
    for record in eftu_records:
        clean_seq = str(record.seq).replace('-','')
        if len(clean_seq) < 200:
            print('This should not be happening...')
            continue
        if record.id[:5] in valid_species:
            outfile.write('>{}\n'.format(record.id))
            outfile.write('{}\n'.format(clean_seq))

# Combine these cleaned files into one large `fasta` file

In [8]:
if2_records = list(SeqIO.parse('../Data/AJH/Orthologs/OMAGroup_IF2.clean.fa', 'fasta'))
eftu_records = list(SeqIO.parse('../Data/AJH/Orthologs/OMAGroup_EFTU.clean.fa', 'fasta'))

with open('../Data/AJH/Orthologs/OMAGroup_IF2_EFTU_combined.fa', 'w') as outfile:
    for record in if2_records:
        clean_seq = str(record.seq)
        outfile.write('>{}\n'.format(record.id+'_if2')) ###adding a little tag for my sanity
        outfile.write('{}\n'.format(clean_seq))
    for record in eftu_records:
        clean_seq = str(record.seq)
        outfile.write('>{}\n'.format(record.id+'_eftu')) ###adding a little tag for my sanity
        outfile.write('{}\n'.format(clean_seq))

# Sequence alignment

There are a few strategies here, none of which is best because... well... we simply don't know!

First, note that it's unnecessary (but possible) to run MAFFT in python. E.g.:

```
import subprocess
fa_file = '../Data/AJH/IF2_EFTU_combined.fa'
aligned_file = '../Data/AJH/combined_fast.aln'
subprocess.call('mafft {} > {}'.format(fa_file, aligned_file), shell=True)
```

For the full alignment, I tried using these three options:
```
mafft --retree 2 --maxiterate 1000 IF2_EFTU_combined.fa > combined_fast.aln
mafft --localpair --maxiterate 1000 IF2_EFTU_combined.fa > combined_linsi.aln
mafft --globalpair --maxiterate 1000 IF2_EFTU_combined.fa > combined_ginsi.aln
```

I also took what (I think?) is a better approach and first aligned each family, then used these individual alignments as seeds for a full alignment.
```
mafft --retree 2 --maxiterate 1000 IF2_OMAGroup.clean.fa > IF2_fast.aln
mafft --retree 2 --maxiterate 1000 EFTU_OMAGroup.clean.fa > EFTU_fast.aln
mafft --maxiterate 1000 --seed IF2_fast.aln --seed EFTU_fast.aln /dev/null > combined_group_fast.aln

mafft --localpair --maxiterate 1000 IF2_OMAGroup.clean.fa > IF2_linsi.aln
mafft --localpair --maxiterate 1000 EFTU_OMAGroup.clean.fa > EFTU_linsi.aln
mafft --maxiterate 1000 --seed IF2_linsi.aln --seed EFTU_linsi.aln /dev/null > combined_group_linsi.aln

mafft --globalpair --maxiterate 1000 IF2_OMAGroup.clean.fa > IF2_ginsi.aln
mafft --globalpair --maxiterate 1000 EFTU_OMAGroup.clean.fa > EFTU_ginsi.aln
mafft --maxiterate 1000 --seed IF2_ginsi.aln --seed EFTU_ginsi.aln /dev/null > combined_group_ginsi.aln
```

So in the end I have a total of 6 distinct alignments! Which is best? Who knows.
```
combined_fast.aln
combined_linsi.aln
combined_ginsi.aln
combined_group_fast.aln
combined_group_linsi.aln
combined_group_ginsi.aln
```

**It's really unclear to me at the moment how we quantify the *succss* of an alignment or rather, whether there is evidence of true homology or not**

# Tree construction 

Ideally, the next step would be to make some trees from these alignments. Lots of options here and I've only begun to explore the range of possibilities. We have time and the alignments aren't huge so I think we can be accurate (e.g. IQTREE). What parameters? Well... maybe...:

```
iqtree -s /PATH/TO/MY/FILE.ALN -m LG+F+R8 -nt INSERT_NUMBER_OF_THREADS_HERE
```

But if we have the time and resources to run the analysis for a day or two, why not double some defaults?:

```
iqtree -s /PATH/TO/MY/FILE.ALN -m LG+F+R8 -nt INSERT_NUMBER_OF_THREADS_HERE -ninit 200 -ntop 40 -nbest 10 -nstop 200 -allnni
```

Or perform rapid bootstraping?:
```
iqtree -s /PATH/TO/MY/FILE.ALN -m LG+F+R8 -nt INSERT_NUMBER_OF_THREADS_HERE -bb 1000
```

Since these are single copy orthologs we could of course try to construct a species tree from the concatenated set (perhaps including a few other core gene orthologs).

A nice thing about `IQTREE` is that we could just use it to identify the ancestral sequences at each node as well (rather than resorting to PAML)

# Next steps

The thing that I'm not clear on when it comes to sequence-level analyses here is what we *do* with some of this data. So the two protein families can be aligned, what does that tell us? I'm not sure what random expectation is here. 

One possibility for future work is to perform direct coupling analysis on the `eggNOG` files (after aligning them). They each contain a few thousand sequences so it would be really easy enough to perform DCA on each file independently (and here we don't care who is or isn't a direct ortholog). I'm wondering if we could then kind of "align" the residue-residue coupling matrices. But again, what's a good alignment and what's a bad alignment? This would be a way to say how similar the individual matrices are to one-another but I don't quite know what the null control would be. 

Probably the easiest thing would be to get coupling matrices, then limit each matrix based on a species (like *E. coli*). Basically, for the IF2 matrix select only the sites that different possible sequence alignments (from above) say are aligned with EF-TU residues. Then do the same for the EF-TU residues and you should have two matrices that are the exact same size all with valid entries. One could score the quality of the different alignments based on how well their coupling matrices agree. This would select the best possible alignment, but not sure where to take that.

Obvious null-model controls are to find other ortholog groups / proteins that appear in each of these species! Then do the full pipeline to get like an expected null distribution. But what a pain...